# 신뢰도 가중을 이용한 CF

- CF의 정확도를 개선할 수 있는 방법으로는 신뢰도 가중(significance weighting)이 있다.
- 공통 아이템이 많은 사용자와의 유사도에 공통 아이템이 적은 사용자와의 유사도보다 더 큰 가중치를 주자는 것이다. 그런데 에측값은 매우 민감해서 계산식을 약간만 바꿔도 RMSE 값이 크게 변동하기 때문에 공통 아이템의 수를 가중치로 직접 사용하면 얻는 것보다 잃는 것이 클 가능성이 높다. 즉, 신뢰도(공통으로 평가한 아이템의 수)가 일정값 이상인 사용자만 이웃 사용자로 활용하는 것이다 

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('./data/u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('./data/u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./data/u.data', sep='\t', names=r_cols, encoding='latin-1')

# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)
# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

# train, test 데이터 분리
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

# 정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수 
def score(model, neighbor_size=0):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs]) # change
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

In [3]:
# train set 사용자들의 Cosine similarities 계산
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

In [4]:
# train 데이터의 user의 rating 평균과 영화의 평점편차 계산 
rating_mean = rating_matrix.mean(axis=1)
rating_bias = (rating_matrix.T - rating_mean).T

###  사용자별 공통 평가 수 계산 

In [5]:
rating_binary1 = np.array((rating_matrix > 0).astype(float)) # 전체 full matrix 중에 평점이 있는 경우만 1로 표시 
rating_binary2 = rating_binary1.T # 위에서 구한 행렬의 전치행렬(transposed matrix)

# 두 형렬의 내적 연산(dot)을 하면 그 결과 행렬의 각 원소는 각각의 사용자가 공통으로 평가한 영화의 수가 되고,
# 대각선은 각 사용자가 평가하나 영화의 수가 된다 
counts = np.dot(rating_binary1, rating_binary2) 
counts = pd.DataFrame(counts, index=rating_matrix.index, columns=rating_matrix.index).fillna(0) # 구한 공통 평가 수를 pandas DataFrame으로 변환
counts

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,204.0,14.0,3.0,4.0,44.0,49.0,82.0,19.0,3.0,44.0,...,44.0,8.0,26.0,13.0,22.0,10.0,28.0,7.0,15.0,45.0
2,14.0,47.0,6.0,3.0,2.0,20.0,12.0,3.0,3.0,6.0,...,7.0,8.0,19.0,10.0,15.0,6.0,8.0,6.0,4.0,4.0
3,3.0,6.0,40.0,6.0,0.0,6.0,10.0,4.0,0.0,4.0,...,1.0,0.0,10.0,4.0,6.0,1.0,8.0,2.0,4.0,1.0
4,4.0,3.0,6.0,18.0,2.0,5.0,5.0,1.0,1.0,1.0,...,3.0,1.0,5.0,4.0,2.0,1.0,4.0,1.0,5.0,3.0
5,44.0,2.0,0.0,2.0,131.0,20.0,62.0,12.0,3.0,17.0,...,34.0,3.0,11.0,2.0,12.0,5.0,17.0,2.0,9.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,10.0,6.0,1.0,1.0,5.0,8.0,8.0,2.0,1.0,4.0,...,2.0,12.0,11.0,4.0,17.0,37.0,4.0,3.0,1.0,7.0
940,28.0,8.0,8.0,4.0,17.0,33.0,35.0,13.0,2.0,26.0,...,25.0,4.0,13.0,7.0,10.0,4.0,80.0,4.0,14.0,16.0
941,7.0,6.0,2.0,1.0,2.0,9.0,5.0,4.0,2.0,4.0,...,2.0,2.0,8.0,5.0,7.0,3.0,4.0,17.0,2.0,4.0


In [6]:
def CF_knn_bias_sig(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_bias:
        sim_scores = user_similarity[user_id].copy() 
        movie_ratings = rating_bias[movie_id].copy() 
        no_rating = movie_ratings.isnull() # 현 movie에 대한 rating이 없는 사용자 표시 (현재 영화에 대해서 평가하지 않은 사용자 True)
        common_counts = counts[user_id] # 현재 사용자와 다른 사용자간 공통 평가 아이템 수 가져오기 
        low_significance = common_counts < SIG_LEVEL # 공통으로 평가한 영화의 수가 SIG_LEVEL보다 낮은 사용자 표시(True)
        none_rating_idx = movie_ratings[no_rating | low_significance].index # 평가를 안 하였거나, SIG_LEVEL이 기준 이하인 user 제거 
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx) 
        if neighbor_size == 0:          
            mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum() 
        else:                       
            if len(sim_scores) > MIN_RATINGS:
                '''
                현재 영화를 평가한 사용자 수가 미리 정한 최소 사용자 수(MIN_RATINGS)보다 큰 경우만 예측값을 계산하고,
                그렇지 않은 경우에는 기본값(현 사용자평균)을 예측값으로 한다 
                '''
                neighbor_size = min(neighbor_size, len(sim_scores)) 
                sim_scores = np.array(sim_scores) 
                movie_ratings = np.array(movie_ratings)
                user_idx = np.argsort(sim_scores) 
                sim_scores = sim_scores[user_idx][-neighbor_size:] 
                movie_ratings = movie_ratings[user_idx][-neighbor_size:] 
                # 편차로 예측치 계산
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                # 예측값에 현 사용자의 평균 더하기
                prediction = prediction + rating_mean[user_id]  
            else:
                prediction = rating_mean[user_id]  
    else:
        prediction = rating_mean[user_id] 
    return prediction  

In [7]:
SIG_LEVEL = 3 # 최소 신뢰도(공통 평가 영화 수)를 3으로 한다 
MIN_RATINGS = 2 # 최소 사용자 수(현재 영화를 평가한 사용자 수)를 2로 한다
score(CF_knn_bias_sig, 30) # 신뢰도 가중 방법과 이웃 크기 30으로 CF 정확성 계산 

0.9448223422142998